In [14]:
import openai
from youtube_transcript_api import YouTubeTranscriptApi
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, StorageContext, Document
from llama_index.core import (
    load_index_from_storage,
    load_indices_from_storage,
    load_graph_from_storage,
)
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core.memory import ChatMemoryBuffer
import os
import openai
client = OpenAI()
from youtube_transcript_api import YouTubeTranscriptApi
from urllib.parse import urlparse, parse_qs



# Function to extract video ID from YouTube URL
def extract_video_id(url):
    query = urlparse(url)
    if query.hostname == 'youtu.be':
        return query.path[1:]
    if query.hostname in ('www.youtube.com', 'youtube.com'):
        if query.path == '/watch':
            return parse_qs(query.query)['v'][0]
        if query.path[:7] == '/embed/':
            return query.path.split('/')[2]
        if query.path[:3] == '/v/':
            return query.path.split('/')[2]
    return None

# Function to fetch YouTube transcript
def get_youtube_transcript(video_url):
    video_id = extract_video_id(video_url)
    if video_id:
        try:
            transcript = YouTubeTranscriptApi.get_transcript(video_id)
            full_transcript = " ".join([t['text'] for t in transcript])
            return full_transcript
        except Exception as e:
            return f"Error: {str(e)}"
    return "Invalid URL"

# Fetch YouTube transcript
video_url = "https://www.youtube.com/watch?v=aQDOU3hPci0"  # Replace with your video URL
document = get_youtube_transcript(video_url)
#save as a file
with open("transcript.txt", "w") as file:
    file.write(document)



#create rag agent
llm = OpenAI(model="gpt-4o", temperature=0)


documents = [Document(text=document , extra_info={})]
index = VectorStoreIndex.from_documents(documents)

memory = ChatMemoryBuffer.from_defaults()
chat_engine = index.as_chat_engine(
    chat_mode="context",
    memory=memory,
    system_prompt="""
    Imagine you're a friendly teacher who's here to help. 
    When you answer questions, keep it casual and engaging—like you're talking to a 16-year-old student who’s curious to learn more.
    Make sure your explanations are clear and straight to the point, but feel free to keep things light and relatable.
    Avoid using big words or complicated terms unless absolutely necessary, and when you do, break them down into simple, easy-to-understand language.
    Encourage questions and curiosity, and always stay positive and supportive. 
    If things get a little off-topic or inappropriate, kindly guide the conversation back to the subject in a helpful way.
    Use examples and fun analogies to explain tricky concepts, and make sure your answers feel approachable and motivating.
    Remember, you're here to help the student feel confident and excited about learning!

    Rules:
    1. ALWAYS DISPLAY A MESSAGE WHEN UNETHICAL, SEXUALLY EXPLICIT, OR HATE SPEECH IS DETECTED. MESSAGE SHOULD BE IN THE FOLLOWING FORMAT:
    "Hey! I'm sorry, but I can't assist with that. My capabilities are limited to providing information based on the video's content. 
    Please keep the conversation respectful and appropriate. Thank you!"
    2. STRICTLY FOLLOW THE RULES MENTIONED ABOVE.
    """
)

def asktube(query):
    response = chat_engine.chat(query)
    return response.response



In [18]:
asktube(input())


'The video is about the concept of growth mindset and how our brains have the capacity to change and grow throughout our lives. It discusses how adopting a growth mindset, which is the belief that our abilities are not fixed but can be developed through effort and learning, can lead to positive outcomes in various aspects of life. The video also touches on how stress can actually enhance performance and how understanding these concepts can have a powerful impact on our self-regard, learning, and overall success. It provides insights and tools based on scientific research to help build a growth mindset and leverage stress for better performance.'

In [7]:
documents 


[Document(id_='f362a000-d3c7-482a-9055-a3919d695e96', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text="[Music] we're no strangers to love you know the rules and so do I I full commitments while I'm thinking of you wouldn't get this from any other guy I just want to tell you how I'm feeling got to make you understand Never Going To Give You Up never going to let you down never going to run around and desert you never going to make you cry never going to say goodbye never going to tell a lie and hurt you we've known each other for so long your heart's been aching but your to sh to say it inside we both know what's been going on we know the game and we're going to playing and if you ask me how I'm feeling don't tell me you're too my you see Never Going To Give You Up never going to let you down never to run around and desert you never going to make you cry never going to say goodbye never going to tell a lie and hurt you 